In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC, LinearSVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, log_loss
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.calibration import CalibratedClassifierCV
from sklearn.externals import joblib
import pickle
import jieba

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def save_model(Vectorizer, Vectorizer_filename, Model, Model_filename):
    print('Save model...')
    print(joblib.dump(Vectorizer, 'model/' + Vectorizer_filename + '.model'))
    print(joblib.dump(Model, 'model/'+ Model_filename +'.model'))

In [3]:
df_train = pd.read_csv('data/train_after_preprocess.csv')
df_train.head()

,ID,review,label
0,1,jo bhi ap se tou behtar hoon,Negative
1,2,ya allah meri sister affia ki madad farma,Positive
2,3,yeh khud chahta umar main shadi krna ogi,Negative
3,4,tc apky mun xe exe alfax achy nae lgty,Negative
4,5,good,Positive


In [4]:
print(len(df_train['review']))
print(type(df_train['review']))

6328
<class 'pandas.core.series.Series'>


In [5]:
df_test = pd.read_csv("data/test_after_preprocess.csv")
df_test.head()

,ID,review
0,1,phr tissuw se saaf
1,2,jail road per firing se shakhs janbahaq
2,3,mehfil loot li aunty ne
3,4,rehnay butt sahb nay galiya boht deni hain
4,5,zabardast


In [6]:
print(len(df_test['review']))
type(df_test['review'])

2712


pandas.core.series.Series

In [9]:
X = np.array([review for review in df_train['review']])
Y = np.array([1 if label == 'Positive' else 0 for label in df_train['label']])
X_test = np.array([review for review in df_test['review']])
X_all = list(np.concatenate((X, X_test), axis = 0))

In [10]:
X_all = [word_tokenize(sent) for sent in X_all]

In [29]:
feature_num = 100

In [11]:
import gensim

model = gensim.models.Word2Vec(X_all,min_count =5,window =8,size=feature_num)   # X是经分词后的文本构成的list，也就是tokens的列表的列表
embeddings_index = dict(zip(model.wv.index2word, model.wv.vectors))

print('Found %s word vectors.' % len(embeddings_index))

Found 3550 word vectors.


In [39]:
def sent2vec(sent):
    M = []
    words_list = word_tokenize(sent)
    for word in words_list:
        try:
            M.append(model[word])
        except:
            continue
    M = np.array(M)
    vec = np.sum(M, axis = 0)
    return vec / np.sqrt((vec**2).sum())

In [40]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.3, shuffle=True)
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)

(4429,)
(4429,)
(1899,)
(1899,)


In [41]:
X_train_w2v = [sent2vec(sent) for sent in tqdm(X_train)]
X_valid_w2v = [sent2vec(sent) for sent in tqdm(X_valid)]
X_test_w2v = [sent2vec(sent) for sent in tqdm(X_test)]



  0%|          | 0/4429 [00:00<?, ?it/s]/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


  1%|▏         | 62/4429 [00:00<00:07, 618.65it/s]

  2%|▏         | 107/4429 [00:00<00:07, 556.10it/s]

  4%|▍         | 178/4429 [00:00<00:07, 594.52it/s]

  6%|▌         | 259/4429 [00:00<00:06, 645.26it/s]

  7%|▋         | 329/4429 [00:00<00:06, 660.49it/s]

 10%|▉         | 424/4429 [00:00<00:05, 726.19it/s]

 12%|█▏        | 536/4429 [00:00<00:04, 811.77it/s]

 15%|█▌        | 673/4429 [00:00<00:04, 924.13it/s]

 18%|█▊        | 778/4429 [00:00<00:03, 958.15it/s]

 20%|█▉        | 879/4429 [00:01<00:03, 938.97it/s]

 22%|██▏       | 978/4429 [00:01<00:03, 952.19it/s]

 

In [42]:
X_train_w2v = np.array(X_train_w2v)
print(X_train_w2v.shape)

(4429,)


In [ ]:
clf =  CalibratedClassifierCV(LinearSVC(), cv = 5) 
clf.fit(X_train_w2v, Y_train)